In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from bci_dataset import BCIDataset

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, WindowOperations, NoiseTypes

In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cpu')

In [3]:
class BCIModel(nn.Module):
    def __init__(self, classes=3):
        super(BCIModel, self).__init__()
        
        self.conv1 = nn.Conv1d(8, 64, 1)
        self.conv2 = nn.Conv1d(64, 256, 1)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, 16, 1)
        
        self.fc1 = nn.Linear(16*256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        x = torch.flatten(x, 1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [13]:
train_dataset = BCIDataset('raw_data_5min.npy')
test_dataset = BCIDataset('raw_data_test.npy')

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [14]:
model = BCIModel()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.SmoothL1Loss()

In [15]:
def fwd_pass(inputs, targets, train=False):
  inputs = inputs.to(device)
  targets = targets.to(device)

  if train:
    model.zero_grad()
  
  outputs = model(inputs)
  matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, targets)]
  acc = matches.count(True) / len(matches)
  loss = loss_function(outputs, targets)

  if train:
    loss.backward()
    optimizer.step()
  
  return acc, loss

In [16]:
writer = SummaryWriter()

EPOCHS = 30
global_step = 0
for epoch in range(EPOCHS):
    progress_bar = tqdm(total=len(train_dataloader))
    progress_bar.set_description(f"Epoch {epoch + 1}/{EPOCHS}")
    
    for step, batch in enumerate(train_dataloader):
        inputs = batch['inputs']
        targets = batch['targets']
        
        acc, loss = fwd_pass(inputs, targets, train=True)
        loss = loss.detach().item()
        
        progress_bar.update(1)
        logs = {"loss": loss, "accuracy": acc, "step": step}
        progress_bar.set_postfix(**logs)
        
        writer.add_scalar('Loss/train', loss, global_step)
        writer.add_scalar('Accuracy/train', acc, global_step)
        
        
        global_step += 1
        
    
    
    batch = next(iter(test_dataloader))
    inputs = batch['inputs']
    targets = batch['targets']
        
    test_acc, test_loss = fwd_pass(inputs, targets, train=False)
        
    writer.add_scalar('Loss/test', test_acc, global_step)
    writer.add_scalar('Accuracy/test', test_loss, global_step)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 